In [1]:
import configparser

import pandas as pd
from binance import Client

CFG_FL_NAME = "../user.cfg"
USER_CFG_SECTION = "binance_user_config"
config = configparser.ConfigParser()
config.read(CFG_FL_NAME)

api_key = config.get(USER_CFG_SECTION, "api_key")
api_secret = config.get(USER_CFG_SECTION, "api_secret_key")
client = Client(api_key, api_secret)

print("Client created", client)

Client created <binance.client.Client object at 0x0000017F55E755B0>


In [2]:
from datetime import datetime
from pandas import json_normalize

start_from = "23:16:35 08/11/2022"
symbol = 'SOLUSDT'

start_from_timestamp = int(datetime.strptime(start_from, "%H:%M:%S %d/%m/%Y").timestamp())*1000


def download_spot_trading_history_for_pair(pair: str, frm: int):
    trades = client.get_my_trades(symbol=pair, startTime=frm)
    trades_df = json_normalize(trades)
    df = trades_df.query('isBuyer == True')
    df['timestamp'] = pd.to_datetime(df['time'], unit='ms')
    df[['price','qty','quoteQty']] =  df[['price','qty','quoteQty']].apply(lambda x:pd.to_numeric(x,errors='coerce'))
    df.sort_values(by='time', ascending=False, inplace=True)
    return pd.DataFrame(df, columns=['price', 'qty', 'quoteQty', 'time','timestamp'])


t = download_spot_trading_history_for_pair(symbol, start_from_timestamp)

t

,price,qty,quoteQty,time,timestamp
1,15.87,0.25,3.9675,1668009471177,2022-11-09 15:57:51.177
2,15.87,0.53,8.4111,1668009471177,2022-11-09 15:57:51.177
0,23.80,10.16,241.8080,1667949395876,2022-11-08 23:16:35.876


In [3]:
t.dtypes

price               float64
qty                 float64
quoteQty            float64
time                  int64
timestamp    datetime64[ns]
dtype: object

In [4]:
## now figure out avg cost of purchase

total_spent = t.loc[t['time'] >= start_from_timestamp, 'quoteQty'].sum()

total_coin = t.loc[t['time'] >= start_from_timestamp, 'qty'].sum()

avg_price_per_coin = total_spent / total_coin

print(f'total_spent: {total_spent} , total_coin : {total_coin} , avg price : {avg_price_per_coin}')

total_spent: 254.1866 , total_coin : 10.94 , avg price : 23.234606946983547


In [5]:
import ipywidgets as widgets

target_avg_price_per_coin = 11

all_coin_price = client.get_symbol_ticker()
current_price_per_coin = float(list(filter(lambda x: x['symbol'] == symbol, all_coin_price))[0]['price'])


def get_avg_price(total_spent_amount: float, amount_of_coins: float) -> float:

    __total_spent = total_spent + float(total_spent_amount)
    __total_coins = float(amount_of_coins) + total_coin
    return __total_spent / __total_coins


def get_total_coins(spent_amount):
    return float(spent_amount / current_price_per_coin)


total_purchase_slider = widgets.IntSlider(min=0, max=500, value=100)
total_number_of_coins = widgets.Label(value=f'{get_total_coins(total_purchase_slider.value)}')
total_avg_value = widgets.Label(value=f'{get_avg_price(total_purchase_slider.value, total_number_of_coins.value)}')


def on_value_changed(change):
    tc = get_total_coins(change['new'])
    total_number_of_coins.value = str(tc)
    total_avg_value.value = str(get_avg_price(change['new'], tc))


total_purchase_slider.observe(on_value_changed, names='value')

target_box = widgets.HBox([widgets.Label(value='Target Price'), widgets.Label(value=f'{target_avg_price_per_coin}')])
current_price = widgets.HBox(
    [widgets.Label(value='Current Price per coin'), widgets.Label(value=f'{current_price_per_coin}')])

purchase_amount = widgets.HBox([widgets.Label(value='Purchase Amount (USD)'), total_purchase_slider])
purchased_coins = widgets.HBox([widgets.Label(value=f'Purchased Coins ({symbol})'), total_number_of_coins])
total_avg = widgets.HBox([widgets.Label(value=f'Total Avg'), total_avg_value])

w = widgets.VBox([target_box, current_price, purchase_amount, purchased_coins, total_avg, ])
display(w)


In [8]:
__total_spent = total_spent + float(total_purchase_slider.value)
__total_coins = get_total_coins(float(total_purchase_slider.value)) + total_coin

print(__total_spent)
print(__total_coins)

509.1866
37.25578947368421


In [31]:
import sqlite3


def copy_sqlite_table_rows_between_two_database(from_db: str, to_db: str, table_name: str , insert_command:str):
    '''
    Copy sqlite table rows between two database
    '''
    print(f'Copying table : {table_name}')
    from_conn = sqlite3.connect(from_db)
    to_conn = sqlite3.connect(to_db)
    from_conn.row_factory = sqlite3.Row
    cur = from_conn.cursor()
    cur.execute("SELECT * FROM %s" % table_name)
    rows = cur.fetchall()
    
    to_cursor = to_conn.cursor()
    to_cursor.execute("DELETE FROM " + table_name)
    to_conn.commit()
    
    to_conn.executemany(insert_command % table_name, rows)
    to_conn.commit()
    from_conn.close()
    to_conn.close()


from_db = "C:/AgodaGit/binance-trade-bot-on-nexus-sync/data1/crypto_trading.db"
to_db = "C:/AgodaGit/binance-trade-bot/data/crypto_trading.db"


copy_sqlite_table_rows_between_two_database(
    from_db,
    to_db,
    'pairs',
    "INSERT INTO %s VALUES (?,?,?,?)"
)

# from_conn = sqlite3.connect(from_db)
# from_cursor = from_conn.cursor()
# from_cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
# from_tables = from_cursor.fetchall()
# from_conn.close()

# for i in from_tables:
#     
#     copy_sqlite_table_rows_between_two_database(
#         from_db,
#         to_db,
#         i[0]
#     )






Copying table : pairs


In [32]:
import sys

sys.path.append('../')

import os
import sqlite3
from configparser import ConfigParser
from telegrambot.btb_manager_telegram.table import tabularize
from telegrambot.btb_manager_telegram.utils import i18n_format, telegram_text_truncator


# def current_ratios(ROOT_PATH=None):
#     logger = Logger()
#     logger.info("Current ratios button pressed.")
# 
#     db_file_path = os.path.join(ROOT_PATH, "data/crypto_trading.db")
#     user_cfg_file_path = os.path.join(ROOT_PATH, "user.cfg")
#     message = [i18n_format("database_not_found", path=db_file_path)]
#     if os.path.exists(db_file_path):
#         try:
#             # Get bridge currency symbol
#             with open(user_cfg_file_path) as cfg:
#                 config = ConfigParser()
#                 config.read_file(cfg)
#                 bridge = config.get("binance_user_config", "bridge")
#                 scout_multiplier = config.get("binance_user_config", "scout_multiplier")
#                 try:  # scout_margin Edgen
#                     scout_margin = (
#                             float(config.get("binance_user_config", "scout_margin")) / 100.0
#                     )
#                     use_margin = config.get("binance_user_config", "use_margin")
#                 except Exception as e:
#                     use_margin = "no"
#                 try:  # scout_margin TnTwist
#                     ratio_calc = config.get("binance_user_config", "ratio_calc")
#                 except Exception as e:
#                     ratio_calc = "default"
#                 if ratio_calc == "scout_margin":
#                     scout_multiplier = float(scout_multiplier) / 100.0
# 
#             con = sqlite3.connect(db_file_path)
#             cur = con.cursor()
# 
#             # Get current coin symbol
#             try:
#                 cur.execute(
#                     """SELECT alt_coin_id FROM trade_history ORDER BY datetime DESC LIMIT 1;"""
#                 )
#                 current_coin = cur.fetchone()[0]
#                 if current_coin is None:
#                     raise Exception()
#             except Exception as e:
#                 logger.error(
#                     f"❌ Unable to fetch current coin from database: {e}", exc_info=True
#                 )
#                 con.close()
#                 return [i18n_format("ratios.db_error")]
# 
#             # Get prices and ratios of all alt coins
#             try:
#                 if use_margin == "yes":  # scout_margin Edgen
#                     logger.info(f"Margin ratio Edgen")
#                     cur.execute(
#                         f"""SELECT sh.datetime, p.to_coin_id, sh.other_coin_price, ((1+0.001*0.001-0.002) * current_coin_price/other_coin_price / sh.target_ratio - 1 - {scout_margin}) AS 'ratio_dict' FROM scout_history sh JOIN pairs p ON p.id = sh.pair_id WHERE p.from_coin_id='{current_coin}' AND p.from_coin_id = ( SELECT alt_coin_id FROM trade_history ORDER BY datetime DESC LIMIT 1) ORDER BY sh.datetime DESC LIMIT ( SELECT count(DISTINCT pairs.to_coin_id) FROM pairs JOIN coins ON coins.symbol = pairs.to_coin_id WHERE coins.enabled = 1 AND pairs.from_coin_id='{current_coin}');"""
#                     )
#                 elif ratio_calc == "scout_margin":  # scout_margin TnTwist
#                     logger.info(f"Margin ratio TnTwist")
#                     cur.execute(
#                         f"""SELECT sh.datetime, p.to_coin_id, sh.other_coin_price, ((1+0.001*0.001-0.002) * current_coin_price/other_coin_price / sh.target_ratio - 1 - {scout_multiplier}) AS 'ratio2_dict' FROM scout_history sh JOIN pairs p ON p.id = sh.pair_id WHERE p.from_coin_id='{current_coin}' AND p.from_coin_id = ( SELECT alt_coin_id FROM trade_history ORDER BY datetime DESC LIMIT 1) ORDER BY sh.datetime DESC LIMIT ( SELECT count(DISTINCT pairs.to_coin_id) FROM pairs JOIN coins ON coins.symbol = pairs.to_coin_id WHERE coins.enabled = 1 AND pairs.from_coin_id='{current_coin}');"""
#                     )
#                 else:  # defaultst
#                     logger.info(f"Margin ratio default")
#                     cur.execute(
#                         f"""SELECT sh.datetime, p.to_coin_id, sh.other_coin_price, ( ( ( current_coin_price / other_coin_price ) - 0.001 * '{scout_multiplier}' * ( current_coin_price / other_coin_price ) ) - sh.target_ratio ) AS 'ratio_dict' FROM scout_history sh JOIN pairs p ON p.id = sh.pair_id WHERE p.from_coin_id='{current_coin}' AND p.from_coin_id = ( SELECT alt_coin_id FROM trade_history ORDER BY datetime DESC LIMIT 1) ORDER BY sh.datetime DESC LIMIT ( SELECT count(DISTINCT pairs.to_coin_id) FROM pairs JOIN coins ON coins.symbol = pairs.to_coin_id WHERE coins.enabled = 1 AND pairs.from_coin_id='{current_coin}');"""
#                     )
#                 query = cur.fetchall()
# 
#                 # Generate message
#                 last_update = datetime.strptime(query[0][0], "%Y-%m-%d %H:%M:%S.%f")
#                 query = sorted(query, key=lambda k: k[-1], reverse=True)
# 
#                 m_list = [
#                     f"\n{i18n_format('ratios.last_update', update=last_update.strftime('%H:%M:%S %d/%m/%Y'))}\n\n"
#                     f"{i18n_format('ratios.compared_ratios', coin=current_coin)}\n"
#                 ]
#                 max_length_ticker = max([len(i[1]) for i in query] + [4])
# 
#                 m_list.extend(
#                     tabularize(
#                         [
#                             i18n_format("ratios.coin"),
#                             i18n_format("ratios.price", bridge=bridge),
#                             i18n_format("ratios.ratio"),
#                         ],
#                         [
#                             [c[1] for c in query],
#                             [c[2] for c in query],
#                             [c[3] for c in query],
#                         ],
#                         [6, 12, 12],
#                         align="left",
#                         add_spaces=[True, True, False],
#                     )
#                 )
# 
#                 message = telegram_text_truncator(m_list)
#                 con.close()
#             except Exception as e:
#                 logger.error(
#                     f"❌ Something went wrong, unable to generate ratios at this time: {e}",
#                     exc_info=True,
#                 )
#                 con.close()
#                 return [
#                     i18n_format("ratios.gen_error"),
#                     i18n_format("logging_enabled_error"),
#                 ]
#         except Exception as e:
#             logger.error(
#                 f"❌ Unable to perform actions on the database: {e}", exc_info=True
#             )
#             message = [i18n_format("ratios.db_error")]
#     return message

def current_ratios(ROOT_PATH:str = None):
    logger = Logger()
    logger.info("Next coin button pressed.")

    db_file_path = os.path.join(ROOT_PATH, "data/crypto_trading.db")
    user_cfg_file_path = os.path.join(ROOT_PATH, "user.cfg")
    message = [f"{i18n_format('database_not_found', path=db_file_path)}"]
    if os.path.exists(db_file_path):
        try:
            # Get bridge currency symbol
            with open(user_cfg_file_path) as cfg:
                config = ConfigParser()
                config.read_file(cfg)
                bridge = config.get("binance_user_config", "bridge")
                scout_multiplier = config.get("binance_user_config", "scout_multiplier")
                try:  # scout_margin Edgen
                    scout_margin = (
                            float(config.get("binance_user_config", "scout_margin")) / 100.0
                    )
                    use_margin = config.get("binance_user_config", "use_margin")
                except Exception as e:
                    use_margin = "no"
                try:  # scout_margin TnTwist
                    ratio_calc = config.get("binance_user_config", "ratio_calc")
                except Exception as e:
                    ratio_calc = "default"
                if ratio_calc == "scout_margin":
                    scout_multiplier = float(scout_multiplier) / 100.0

            con = sqlite3.connect(db_file_path)
            cur = con.cursor()

            # Get prices and percentages for a jump to the next coin
            try:
                if use_margin == "yes":  # scout_margin Edgen
                    logger.info(f"Margin ratio Edgen")
                    cur.execute(
                        f"""SELECT p.to_coin_id as other_coin, sh.other_coin_price, (1-0.001*0.001-0.002) * current_coin_price / (sh.target_ratio *(1+{scout_margin})) AS 'price_needs_to_drop_to', (1-0.001*0.001-0.002) * current_coin_price / (sh.target_ratio *(1+{scout_margin})) / sh.other_coin_price as 'percentage'  FROM scout_history sh JOIN pairs p ON p.id = sh.pair_id WHERE p.from_coin_id = (SELECT alt_coin_id FROM trade_history ORDER BY datetime DESC LIMIT 1) ORDER BY sh.datetime DESC, percentage DESC LIMIT (SELECT count(DISTINCT pairs.to_coin_id) FROM pairs JOIN coins ON coins.symbol = pairs.to_coin_id WHERE coins.enabled = 1 AND pairs.from_coin_id=(SELECT alt_coin_id FROM trade_history ORDER BY datetime DESC LIMIT 1));"""
                    )
                elif ratio_calc == "scout_margin":  # scout_margin TnTwist
                    logger.info(f"Margin ratio TnTwist")
                    cur.execute(
                        f"""SELECT p.to_coin_id as other_coin, sh.other_coin_price, (1-0.001*0.001-0.002) * current_coin_price / (sh.target_ratio *(1+{scout_multiplier})) AS 'price_needs_to_drop_to', (1-0.001*0.001-0.002) * current_coin_price / (sh.target_ratio *(1+{scout_multiplier})) / sh.other_coin_price as 'percentage'  FROM scout_history sh JOIN pairs p ON p.id = sh.pair_id WHERE p.from_coin_id = (SELECT alt_coin_id FROM trade_history ORDER BY datetime DESC LIMIT 1) ORDER BY sh.datetime DESC, percentage DESC LIMIT (SELECT count(DISTINCT pairs.to_coin_id) FROM pairs JOIN coins ON coins.symbol = pairs.to_coin_id WHERE coins.enabled = 1 AND pairs.from_coin_id=(SELECT alt_coin_id FROM trade_history ORDER BY datetime DESC LIMIT 1));"""
                    )
                else:  # default
                    logger.info(f"Margin ratio default")
                    cur.execute(
                        f"""
                        SELECT p.to_coin_id                                                                               as other_coin,
                               sh.other_coin_price,
                               
                               (current_coin_price - 0.001 * '{scout_multiplier}' * current_coin_price) / sh.target_ratio AS 'price_needs_to_drop_to',
                               
                               ((current_coin_price - 0.001 * '{scout_multiplier}' * current_coin_price) / sh.target_ratio) / sh.other_coin_price  as 'percentage'
                        
                        FROM scout_history sh
                                 JOIN pairs p ON p.id = sh.pair_id
                        WHERE p.from_coin_id = (SELECT alt_coin_id FROM trade_history ORDER BY datetime DESC LIMIT 1) --- get the current coin by taking the last row of the trade_history table
                        ORDER BY sh.datetime DESC, percentage DESC
                        LIMIT (SELECT count(DISTINCT pairs.to_coin_id)
                               FROM pairs
                                        JOIN coins ON coins.symbol = pairs.to_coin_id
                               WHERE coins.enabled = 1
                                 AND pairs.from_coin_id = (SELECT alt_coin_id FROM trade_history ORDER BY datetime DESC LIMIT 1))
                        """
                    )
                query = cur.fetchall()
                m_list = []
                query = sorted(query, key=lambda x: x[3], reverse=True)

                m_list.extend(
                    tabularize(
                        [
                            i18n_format("next_coin.coin"),
                            i18n_format("next_coin.percentage"),
                            i18n_format("next_coin.current_price"),
                            i18n_format("next_coin.target_price"),
                        ],
                        [
                            [c[0] for c in query],
                            [str(round(c[3] * 100, 2)) for c in query],
                            [c[1] for c in query],
                            [c[2] for c in query],
                        ],
                        [6, 7, 8, 8],
                        add_spaces=[True, True, False, False],
                        align=["center", "left", "left", "left"],
                    )
                )

                message = telegram_text_truncator(m_list)
                con.close()
            except Exception as e:
                logger.error(
                    f"❌ Something went wrong, unable to generate next coin at this time: {e}",
                    exc_info=True,
                )
                con.close()
                return [
                    i18n_format("next_coin.error"),
                    i18n_format("logging_enabled_error"),
                ]
        except Exception as e:
            logger.error(
                f"❌ Unable to perform actions on the database: {e}", exc_info=True
            )
            message = [i18n_format("next_coin.db_error")]
    return message

x = current_ratios("C:\\AgodaGit\\binance-trade-bot")

print('\n'.join(str(i) for i in x))

2022-12-29 17:34:15,059 - crypto_trading_logger - INFO - Next coin button pressed.
2022-12-29 17:34:15,059 - crypto_trading_logger - INFO - Next coin button pressed.
2022-12-29 17:34:15,059 - crypto_trading_logger - INFO - Next coin button pressed.
2022-12-29 17:34:15,059 - crypto_trading_logger - INFO - Next coin button pressed.
2022-12-29 17:34:15,059 - crypto_trading_logger - INFO - Next coin button pressed.
2022-12-29 17:34:15,059 - crypto_trading_logger - INFO - Next coin button pressed.
2022-12-29 17:34:15,059 - crypto_trading_logger - INFO - Next coin button pressed.
2022-12-29 17:34:15,059 - crypto_trading_logger - INFO - Next coin button pressed.
2022-12-29 17:34:15,059 - crypto_trading_logger - INFO - Next coin button pressed.
2022-12-29 17:34:15,059 - crypto_trading_logger - INFO - Next coin button pressed.
2022-12-29 17:34:15,059 - crypto_trading_logger - INFO - Next coin button pressed.
2022-12-29 17:34:15,059 - crypto_trading_logger - INFO - Next coin button pressed.
2022

```
┌−−−−−−┬−−−−−−−┬−−−−−−−−┬−−−−−−−−┐
│ nex… │ next… │next_co…│next_co…│
├−−−−−−┼−−−−−−−┼−−−−−−−−┼−−−−−−−−┤
│ ADA  │ 46.81 │0.2455  │0.114912│
│ AVAX │ 44.05 │11.16   │4.916314│
│ DOT  │ 43.91 │4.332   │1.902182│
│ XRP  │ 43.46 │0.3452  │0.15004 │
│ BTC  │ 36.1  │16586.23│5987.305│
│ BNB  │ 34.41 │245.8   │84.58308│
│ ETH  │ 33.76 │1197.84 │404.4027│
└−−−−−−┴−−−−−−−┴−−−−−−−−┴−−−−−−−−┘
```



In [ ]:
### calculate ratio and figure out why it does not change !!!!!!
import sys

sys.path.append('../')

from binance_trade_bot.config import Config
from binance_trade_bot.database import Database
from binance_trade_bot.logger import Logger
from binance_trade_bot.auto_trader import AutoTrader
from binance_trade_bot.binance_api_manager import BinanceAPIManager
from binance_trade_bot.strategies import get_strategy
from binance_trade_bot.database import CoinStub

logger = Logger()
config = Config(config_file_path='../user.cfg')
db = Database(logger, config)
manager = BinanceAPIManager.create_manager_paper_trading(config, db, logger, {config.BRIDGE.symbol: 1_000.0})
strategy = get_strategy(config.STRATEGY)
trader: AutoTrader = strategy(manager, db, logger, config)
coinStub = CoinStub.get_by_symbol(symbol)
coin_sell_price = current_price_per_coin
quote_amount = __total_spent
coin_amount = __total_coins
trader._jump_to_best_coin(
    coinStub, coin_sell_price, quote_amount, coin_amount
)


